In [1]:
import json
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

def scrape_booking_hotels(url, hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    hotels = []

    hotel_cards = soup.find_all("div", class_="c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4")
    for card in hotel_cards:
        hotel_name = card.find("div", class_="f6431b446c a15b38c233").text.strip()
        hotel_location = card.find("span", class_="aee5343fdb def9bc142a").text.strip()
        
        # hotel score
        hotel_score_div = card.find('div', class_='a3b8729ab1')
        hotel_score = hotel_score_div.text.strip() if hotel_score_div else None 
        
        # hotel reviews
        hotel_reviews_div = card.find('div', class_='abf093bdfe f45d8e4c32 d935416c47')
        hotel_reviews = hotel_reviews_div.text.strip() if hotel_reviews_div else None     
        
        # Handle hotel price
        hotel_price = card.find('span', class_='f6431b446c fbfd7c1165 e84eb96b1f').text.strip() if card.find('span', class_='f6431b446c fbfd7c1165 e84eb96b1f').text.strip() else None
        
        beds_info = card.find('h4', class_='abf093bdfe e8f7c070a7').text.strip() if card.find('div', class_='abf093bdfe') else None
        rating_div = card.find("div", {"data-testid": "rating-stars", "aria-hidden": "true"})
        rating_span_count = len(rating_div.find_all("span", class_="fcd9eec8fb d31eda6efc c25361c37f")) if rating_div else None

        # Check if the hotel is in the list of hotels with facilities
        pool = 'yes' if hotel_name in hotels_with_pool else 'no'
        restaurant = 'yes' if hotel_name in hotels_with_restaurant else 'no'
        parking = 'yes' if hotel_name in hotels_with_parking else 'no'
        wheelchair = 'yes' if hotel_name in hotels_with_wheelchair else 'no'
        view = 'yes' if hotel_name in hotels_with_view else 'no'


        hotels.append({"Hotel Name": hotel_name, "Address": hotel_location, "Score": hotel_score, "Number of reviews": hotel_reviews, "Price": hotel_price, "Room type": beds_info, "Star rating": rating_span_count, "Room view": view, "Swimming pool": pool, "Restaurant": restaurant, "Parking": parking, "Wheelchair accessibility": wheelchair})

    return hotels

def scrape_and_save_hotels(search_url, hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view):
    all_hotels = []

    try:
        for i in range(1, 28):
            hotels_on_page = scrape_booking_hotels(search_url + "&offset=" + str(i * 25), hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view)
            all_hotels.extend(hotels_on_page)
    except KeyboardInterrupt:
        print("Execution interrupted by user.")

    return all_hotels

def save_to_excel(data, file_path):
    df = pd.DataFrame(data)
    df.to_excel(file_path, index=False)
    print("Data saved to:", file_path)

if __name__ == "__main__":
    # Load hotels with swimming pool from the external Excel file
    hotels_with_pool_df = pd.read_excel('hotels_with_pool.xlsx')
    hotels_with_pool = hotels_with_pool_df['names'].tolist()

    # Load hotels with Restaurant from the external Excel file
    hotels_with_restaurant_df = pd.read_excel('hotels_with_restaurant.xlsx')
    hotels_with_restaurant = hotels_with_restaurant_df['names'].tolist()
 
    # Load hotels with parking from the external Excel file
    hotels_with_parking_df = pd.read_excel('hotels_with_parking.xlsx')
    hotels_with_parking = hotels_with_parking_df['names'].tolist()
    
    # Load hotels with wheel chair from the external Excel file
    hotels_with_wheelchair_df = pd.read_excel('hotels_with_wheelchair.xlsx')
    hotels_with_wheelchair = hotels_with_wheelchair_df['names'].tolist()  
    
    # Load hotels with view from the external Excel file
    hotels_with_view_df = pd.read_excel('hotels_with_view.xlsx')
    hotels_with_view = hotels_with_view_df['names'].tolist()   
    
    search_url = 'https://www.booking.com/searchresults.en-gb.html?label=en-sa-booking-desktop-UmcGSHJHH5d7CxRJ6pHCYgS652796015661%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1012088%3Ali%3Adec%3Adm&sid=d7c87de715469e394879757ca2dd2e2d&aid=2311236&ss=Saudi+Arabia&ssne=Saudi+Arabia&ssne_untouched=Saudi+Arabia&efdco=1&lang=en-gb&src=searchresults&dest_id=186&dest_type=country&checkin=2024-06-03&checkout=2024-06-08&ltfd=1%3A5%3A5-2024_6-2024_7-2024%3A1%3A&group_adults=2&no_rooms=1&group_children=0&nflt=ht_id%3D204'
    
    hotels_data = scrape_and_save_hotels(search_url, hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view)

    excel_file_path = "Saudi_Arabia_hotels.xlsx"
    save_to_excel(hotels_data, excel_file_path)

Data saved to: Saudi_Arabia_hotels.xlsx


In [ ]:
## Data Processing and Data cleaning

##first we are going to